In [11]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/tjwin/Downloads/Tips for Users of Assistive Technology.pdf")

data = loader.load()

c:\Users\tjwin\anaconda3\envs\genai_env_\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000)
docs = text_splitter.split_documents(data)

print("total number of documents: ",len(docs))

total number of documents:  15


In [3]:
docs[0]

Document(page_content='Articulate R ise Accessibility Support  Documentation   \nWhat is this document?  \nOur goal is to develop the most accessible eLearning possible within the design constraints of our \ncurriculum authoring tool, Rise360. For users of Assistive Technology, we have created the following \nAccessibility Support Document to outline some of the behaviors you may observe when completing this \ntraining.  \nKeyboard Navigation Resource  \n• Learners can navigate Rise 360 courses entirely with their keyboards. Th e Rise360 Keyboard \nNavigation Resource article  explains how. And authors don’t have to do anything to enable \nkeyboard accessibility : The first section of this article describes the navigation keys and their \ngeneral behavior. The second section details specifically how to use keyboard navigation in each \nsegment of a course. And the last section explains how to enable tab -key navigation on a Mac.  \nLanding Screen', metadata={'source': 'C:/Users/tjwin/D

In [4]:
from key import gooogle_api_key
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

c:\Users\tjwin\anaconda3\envs\genai_env_\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gooogle_api_key,timeout = 120 )

In [ ]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embedding_model)

In [ ]:
retriver = vectorstore.as_retriever(search_type = "similarity",search_kwags = {"k":10})

k= retriver.invoke("what is the title of document")

In [ ]:
print(k[1].page_content)

In [ ]:
retrieved_docs = retriever.invoke("What kind of services they provide?")

In [ ]:
len(retrieved_docs)

In [9]:
llm = ChatGoogleGenerativeAI(
    google_api_key=gooogle_api_key,  # Fixed variable name
    model="gemini-1.5-pro",
    temperature=0.2,
    #max_output_tokens=500,
    max_tokens = 500
)